In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ Connected to GPU:", gpus[0])
    except RuntimeError as e:
        print(e)
else:
    print("❌ No GPU detected")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
image_datagen = ImageDataGenerator(rescale=1./255)
mask_datagen = ImageDataGenerator(rescale=1./255)

image_generator = image_datagen.flow_from_directory(
    "/content/drive/MyDrive/sem 7/Lab/DL_Lab/Banana_Dataset",
    classes=["images"],
    class_mode=None,
    color_mode='rgb',
    target_size=(128, 128),
    batch_size=8,
    seed=42
)

mask_generator = mask_datagen.flow_from_directory(
    "/content/drive/MyDrive/sem 7/Lab/DL_Lab/Banana_Dataset",
    classes=["mask"],
    class_mode=None,
    color_mode='grayscale',
    target_size=(128, 128),
    batch_size=8,
    seed=42
)


In [ ]:
def combined_generator(image_gen, mask_gen):
    while True:
        img = next(image_gen)
        mask = next(mask_gen)
        yield img, mask

train_generator = combined_generator(image_generator, mask_generator)


In [ ]:
def build_fcnn():
    inputs = Input((128, 128, 3))

    # Encoder
    conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool2)

    # Decoder
    up1 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)

    up2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv5)

    model = Model(inputs, outputs)
    return model

model = build_fcnn()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
steps_per_epoch = len(image_generator)
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20)


In [ ]:
sample_img = next(image_generator)[0]   # first image in batch
sample_mask = next(mask_generator)[0]   # corresponding mask

pred_mask = model.predict(np.expand_dims(sample_img, axis=0))[0]

# Plot results
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(sample_img)

plt.subplot(1, 3, 2)
plt.title("Ground Truth Mask")
plt.imshow(sample_mask.squeeze(), cmap="gray")

plt.subplot(1, 3, 3)
plt.title("Predicted Mask")
plt.imshow(pred_mask.squeeze(), cmap="gray")

plt.show()
